In [6]:
import cv2
import time
import os
from pathlib import Path

import concurrent
import requests

In [7]:
def get_meta(img: cv2.Mat):
    image_data = cv2.imencode('.jpg', img)[1].tobytes()
    urls = [
        
        "https://72c4-34-143-252-179.ngrok.io/useColor",
        "https://52c7-35-247-181-246.ngrok.io/predict"
    ]

    def request_post(url, data):
        return requests.post(url, files={"image": data}).json()

    with concurrent.futures.ThreadPoolExecutor() as executor: # optimally defined number of threads
        res = [executor.submit(request_post, url, image_data) for url in urls]
        concurrent.futures.wait(res)

    return {"color": res[0].result()["predicted"], "model": res[1].result()["predicted"]}

In [8]:
def crop_do (coor, img, conf, cls):
    current_time = int(round(time.time() * 1000))
    print(current_time)
    if not os.path.exists("./runs/"):
        os.makedirs("./runs/")
    Path("./runs/Use1/").mkdir(parents=True, exist_ok=True)
    crop_img = img[ int(coor[1]) : int(coor[3]), int(coor[0]): int(coor[2])]
    #result = cv2.resize(crop_img, (640, 640))
    #result = cv2.cvtColor(result , cv2.COLOR_RGB2GRAY)
    meta = get_meta(crop_img)
    print(meta)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, f'Color: {meta["color"]}', (10,80), font, 3, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(img, f'Model: {meta["model"]}', (10,160), font, 3, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imwrite("./runs/Use1/img" + str(current_time) + ".jpg", img)

In [9]:
from lean_detect import UseModel
veh = UseModel("yolov7.pt", detect_class=[2, 5, 7], confident=0.75)

YOLOR  3736f7b torch 1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191.375MB)



Fusing layers... 


Model Summary: 306 layers, 36905341 parameters, 36905341 gradients, 104.5 GFLOPS


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



In [10]:
veh.detect("./clips/use001.MP4" , do_function=crop_do)

1676910856195
{'color': 'Black', 'model': 'Mazda2'}
1676910878825
{'color': 'White', 'model': 'yaris_old'}
1676910890839
{'color': 'Grey', 'model': 'unknown'}
1676910917750
{'color': 'Bronze', 'model': 'e150'}
1676910956913
{'color': 'Black', 'model': 'unknown'}
1676910972707
{'color': 'Blue', 'model': 'unknown'}
1676910984259
{'color': 'Brown', 'model': 'acv70'}
1676911044339
{'color': 'Black', 'model': 'yaris_mama'}
1676911066751
{'color': 'Black', 'model': 'civic_fc'}
1676911078366
{'color': 'Black', 'model': 'unknown'}
1676911153672
{'color': 'Blue', 'model': 'e160'}
1676911188745
{'color': 'White', 'model': 'rt50'}
1676911225722
{'color': 'Grey', 'model': 'unknown'}
1676911246312
{'color': 'Grey', 'model': 'unknown'}
1676911267000
{'color': 'Grey', 'model': 'unknown'}
1676911281245
{'color': 'Black', 'model': 'Mazda2'}
1676911288581
{'color': 'Black', 'model': 'unknown'}
1676911293145
{'color': 'Black', 'model': 'unknown'}
1676911295670
{'color': 'Brown', 'model': 'unknown'}
16769

# Google storage

In [1]:
! pip install google-cloud

In [2]:
from google.cloud import storage
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r'./carrgclassification-857e3c375cdd.json'

def upload_cs_file(bucket_name, source_file_name, destination_file_name): 
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(destination_file_name)
    blob.upload_from_filename(source_file_name)

In [ ]:
import datetime

def get_cs_file_url(bucket_name, file_name, expire_in=datetime.today() + datetime.timedelta(4000)): 
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    url = bucket.blob(file_name).generate_signed_url(expire_in)

    return url

In [3]:
upload_cs_file('images-bucks', "./DemoCase/img1676910856195.jpg", "test.jpg")